# PGR Data Loader - Two-Phase Workflow

## Overview
This notebook implements a **two-phase data upload** approach:

### **Phase 1: Tenant/State Setup**
- Upload tenant and state information first
- Creates the basic organizational structure
- Template: `Tenant Master.xlsx`

add the boundary as phase 2


### **Phase 2: Common Master Data**
- Select which tenant to configure
- Upload departments, designations, and complaint types
- Template: `Common Master.xlsx`

---

## ⚠️ IMPORTANT: Run Cells in Order!

**You MUST run cells sequentially from top to bottom:**
1. ✅ Initialize Global Variables (Cell 2)
2. ✅ Import Packages (Cell 3)
3. ✅ Download Templates (Cell 4)
4. ✅ Continue with Phase 1...

**Do NOT skip cells or run them out of order!**

---

## templates
- Tenant Master Excel filled with ULB information
- add boundary prerequisted
- Common Master Excel filled with department and complaint data
- API access credentials

---

In [19]:
# Import required packages
import pandas as pd
import json
import warnings
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, FileLink
import os
import sys

warnings.filterwarnings('ignore')

# Initialize Global Variables
# DO NOT skip this cell - Run it first!

CONFIG = {}
CONFIG_SET = False
TENANT_FILE = None
COMMON_MASTER_FILE = None
UPLOADED_TENANTS = []
SELECTED_TENANT = None

# Force reload modules to get latest changes
if 'unified_loader' in sys.modules:
    del sys.modules['unified_loader']
if 'mdms_validator' in sys.modules:
    del sys.modules['mdms_validator']
if 'excel_validator' in sys.modules:
    del sys.modules['excel_validator']

# Import the unified loader module
from unified_loader import UnifiedExcelReader, APIUploader, clean_nans

print("✅ Packages loaded successfully!")
print("✅ Modules reloaded with latest changes!")

✅ Packages loaded successfully!
✅ Modules reloaded with latest changes!


---

#  PHASE 1: TENANT/STATE SETUP

In Phase 1, you will:
1. **Enter State Tenant ID** (e.g., `pg` for Punjab)
2. **Upload Tenant Master Excel** with state and city information
3. **Upload tenant data to API** - Creates state and city tenants

After Phase 1, you'll have a list of uploaded tenants to choose from in Phase 2.

---

In [20]:
# Template Download Links
print("="*70)
print("EXCEL TEMPLATE DOWNLOAD LINKS")
print("="*70)
print("\nClick any link below to download the template:\n")

templates_path = "templates"

# NEW TEMPLATE FILES - Simplified Format
template_files = [
    ("Tenant And branding master.xlsx", "Phase 1: Tenant Info (4 required fields) + Branding"),
    ("Common and Complaint Master.xlsx", "Phase 2: Departments, Designations, and Complaint Types"),
]

# Create clickable download links
html_links = []
for i, (filename, description) in enumerate(template_files, 1):
    filepath = os.path.join(templates_path, filename)
    if os.path.exists(filepath):
        html_links.append(
            f'<div style="margin: 10px 0;">'
            f'<b>{i}.</b> '
            f'<a href="{filepath}" download="{filename}" '
            f'style="font-size: 14px; color: #0066cc; text-decoration: none;">'
            f'{filename}</a> '
            f'<span style="color: #666;">- {description}</span>'
            f'</div>'
        )
    else:
        html_links.append(
            f'<div style="margin: 10px 0;">'
            f'<b>{i}.</b> {filename} <span style="color: red;">(Not found)</span>'
            f'</div>'
        )

# Display as HTML with styling
html_content = f"""
<div style="border: 1px solid #ddd; padding: 15px; border-radius: 5px; background-color: #f9f9f9;">
    <h3 style="margin-top: 0;">📁 NEW Template Files (Simplified Format):</h3>
    {''.join(html_links)}
    <hr style="margin: 20px 0;">
    <p><b>📋 Two-Phase Workflow:</b></p>
    <ol>
        <li><b>Phase 1:</b> Download <code>Tenant And branding master.xlsx</code> - Only 4 required fields!</li>
        <li><b>Phase 2:</b> Download <code>Common and Complaint Master.xlsx</code> - Simplified structure</li>
        <li>Fill the templates (see Read me sheets for instructions)</li>
        <li>Upload them using this notebook</li>
    </ol>
    <p><b>✨ New Features:</b></p>
    <ul>
        <li>✅ Fewer required fields (4 vs 10 in old template)</li>
        <li>✅ ADMIN0/ADMIN1/ADMIN2 hierarchy for international support</li>
        <li>✅ Auto-generates codes (district, region, department, designation)</li>
        <li>✅ Auto-generates localization entries</li>
        <li>✅ Detailed Read me sheets with examples</li>
    </ul>
</div>
"""

display(HTML(html_content))

EXCEL TEMPLATE DOWNLOAD LINKS

Click any link below to download the template:



In [21]:
# Configuration Setup and Tenant Master Upload
# Make sure you've run the "Initialize Global Variables" cell first!

print("="*70)
print("        ⚙️  PHASE 1: CONFIGURATION & UPLOAD TENANT MASTER")
print("="*70)
print()

# Configuration widgets
state_tenant_id = widgets.Text(
    value='pg',
    description='🏛️ State Tenant ID:',
    placeholder='e.g., pg, pb, up',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

# File upload widget
tenant_file_upload = widgets.FileUpload(
    accept='.xlsx,.xls',
    multiple=False,
    description='Select File:',
    layout=widgets.Layout(width='70%')
)

# Combined button
upload_btn = widgets.Button(
    description='✅ Save Config & Upload Tenant Master',
    button_style='success',
    layout=widgets.Layout(width='95%', height='40px')
)

output = widgets.Output()

def on_upload(b):
    global CONFIG, CONFIG_SET, TENANT_FILE, UPLOADED_TENANTS
    
    with output:
        clear_output()
        
        # Validate configuration
        if not state_tenant_id.value.strip():
            print("❌ State Tenant ID required")
            return
        
        # Validate file selection
        if not tenant_file_upload.value:
            print("❌ Please select Tenant Master Excel file")
            return
        
        # Save configuration
        CONFIG = {
            'base_url': 'https://unified-qa.digit.org',
            'state_tenant_id': state_tenant_id.value.strip().lower()
        }
        CONFIG_SET = True
        
        print("="*70)
        print("  ✅ CONFIGURATION SAVED!")
        print("="*70)
        print(f"\n📊 Configuration:")
        print(f"   • State Tenant: {CONFIG['state_tenant_id']}")
        print("="*70)
        
        # Save file
        os.makedirs('upload', exist_ok=True)
        uploaded_file = tenant_file_upload.value[0]
        content = uploaded_file['content']
        filename = 'Tenant_Master.xlsx'
        
        upload_path = os.path.join('upload', filename)
        with open(upload_path, 'wb') as f:
            f.write(content)
        
        print("\n" + "="*70)
        print("  📋 VALIDATING FILE AGAINST MDMS SCHEMA...")
        print("="*70)
        
        # Validate the file against MDMS schema
        try:
            from mdms_validator import MDMSValidator
            mdms_validator = MDMSValidator()
            result = mdms_validator.validate_excel_file(
                excel_file=upload_path,
                tenant_id=CONFIG['state_tenant_id'],
                schema_code='tenant.master'
            )
            
            if result['valid']:
                print("\n✅ MDMS VALIDATION PASSED!")
                print("   All required fields match MDMS schema.")
                TENANT_FILE = upload_path
                
                print("\n" + "="*70)
                print("  ✅ FILE UPLOADED SUCCESSFULLY")
                print("="*70)
                print(f"\n📄 File: {upload_path}")
                print("\n➡️  Proceed to load and upload tenant data")
                print("="*70)
            else:
                print("\n❌ MDMS VALIDATION FAILED!")
                print(f"   Found {len(result['errors'])} error(s):\n")
                
                # Show first 5 errors
                for i, error in enumerate(result['errors'][:5], 1):
                    print(f"   Error {i}:")
                    if 'row' in error:
                        print(f"      Sheet: {error.get('sheet', 'Unknown')}")
                        print(f"      Row: {error['row']}")
                        if 'column' in error:
                            print(f"      Column: {error['column']}")
                        if 'value' in error:
                            print(f"      Value: {error.get('value', 'N/A')}")
                    print(f"      Message: {error['message']}")
                    print()
                
                if len(result['errors']) > 5:
                    print(f"   ... and {len(result['errors']) - 5} more errors")
                
                print("\n⚠️  Please fix the errors in your Excel file and try again.")
                print("="*70)
                TENANT_FILE = None
                CONFIG_SET = False
                
        except Exception as e:
            print(f"\n⚠️  MDMS Validation error: {str(e)}")
            print("   Proceeding without validation...")
            TENANT_FILE = upload_path
            print("\n" + "="*70)
            print("  ✅ FILE UPLOADED SUCCESSFULLY")
            print("="*70)
            print(f"\n📄 File: {upload_path}")
            print("\n➡️  Proceed to load and upload tenant data")
            print("="*70)

upload_btn.on_click(on_upload)

# Display
combined_box = widgets.VBox([
    widgets.HTML("<h3>📋 Step 1.1: Configuration & Upload Tenant Master</h3>"),
    widgets.HTML("<p style='color: #666;'><i>Enter your state tenant ID and upload the Tenant Master Excel file</i></p>"),
    widgets.HTML("<p style='color: #666;'><b>Example:</b> If your state is Punjab, enter <code>pg</code></p>"),
    widgets.HTML("<br>"),
    state_tenant_id,
    widgets.HTML("<p style='font-size: 12px; color: #888;'><i>💡 The Tenant Master Excel will contain cities like pg.citya, pg.cityb under this state tenant</i></p>"),
    widgets.HTML("<br>"),
    widgets.HTML("<p style='color: #666;'><b>Select Tenant Master Excel File:</b></p>"),
    tenant_file_upload,
    widgets.HTML("<p style='font-size: 11px; color: #007bff;'><i>⚡ File will be validated against MDMS schema 'tenant.master'</i></p>"),
    widgets.HTML("<br>"),
    upload_btn,
    output
])

display(combined_box)

        ⚙️  PHASE 1: CONFIGURATION & UPLOAD TENANT MASTER



## Step 1.2: Load and Upload Tenant Data

In [10]:
print("="*70)
print("[PHASE 1] LOADING TENANT DATA")
print("="*70)

if not TENANT_FILE:
    print("❌ Please upload Tenant Master Excel first!")
else:
    # Initialize reader and uploader
    reader = UnifiedExcelReader(TENANT_FILE)
    uploader = APIUploader()
    
    # Read tenant data using NEW format method
    tenants_data, tenants_localization = reader.read_tenant_info()
    
    print(f"\n[INFO] Loaded {len(tenants_data)} tenant(s) from Excel")
    for tenant in tenants_data:
        print(f"   - {tenant['code']}: {tenant['name']} ({tenant['type']})")
        UPLOADED_TENANTS.append(tenant['code'])
    
    print(f"\n[INFO] Auto-generated {len(tenants_localization)} localization entries")
    
    # Upload Tenants
    result_tenants = uploader.create_mdms_data(
        schema_code='tenant.tenants',
        data_list=clean_nans(tenants_data),
        tenant=CONFIG['state_tenant_id'],
        sheet_name='Tenants'
    )
    
    # Upload Localization
    result_loc = uploader.create_localization_messages(
        localization_list=clean_nans(tenants_localization),
        tenant=CONFIG['state_tenant_id'],
        sheet_name='Tenants_Localization'
    )
    print(result_tenants)
    # Summary
    if result_tenants['failed'] == 0:
        print("\n✅ [SUCCESS] Tenants uploaded successfully!")
        print("\n➡️  Proceed to upload State Branding (optional)")
    else:
        print("\n⚠️  [WARNING] Some tenants failed. Check errors/ folder.")

[PHASE 1] LOADING TENANT DATA

[INFO] Loaded 3 tenant(s) from Excel
   - pg: Punjab State (State)
   - pg.citya: City A ULB (City)
   - pg.cityb: City B ULB (City)

[INFO] Auto-generated 3 localization entries

[UPLOADING] tenant.tenants
   Tenant: pg
   Records: 3
   API URL: http://localhost:8094/mdms-v2/v2/_create/{schema_code}
   [EXISTS] [1/3] pg
   [EXISTS] [2/3] pg.citya
   [EXISTS] [3/3] pg.cityb
[SUMMARY] Created: 0
[SUMMARY] Already Exists: 3
[SUMMARY] Failed: 0

[UPLOADING] Localization Messages
   Tenant: pg
   Total Messages: 3
   API URL: http://localhost:8087/localization/messages/v1/_upsert

   Found 1 locales: en_IN
   [ERROR] Locale: en_IN - HTTPConnectionPool(host='localhost', port=8087): Max retries exceeded with url: /localization/messages/v1/_upsert (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x127daae0
[SUMMARY] Created: 0
[SUMMARY] Already Exists: 0
[SUMMARY] Failed: 3

[ERRORS] Found 1 error(s):
   - Locale: en_IN (3 messages)
  

## Step 1.3: Load and Upload State Branding 

In [11]:
print("="*70)
print("[PHASE 1] LOADING STATE BRANDING DATA")
print("="*70)

if not TENANT_FILE:
    print("❌ Please upload Tenant Master Excel first!")
else:
    # Initialize reader and uploader
    reader = UnifiedExcelReader(TENANT_FILE)
    uploader = APIUploader()
    
    # Read branding data using NEW format method
    branding_data = reader.read_tenant_branding()
    
    if not branding_data:
        print("\n[INFO] No branding data found in 'Tenant Branding Deatils' sheet")
        print("   This is optional - you can skip this step if not needed")
    else:
        print(f"\n[INFO] Loaded {len(branding_data)} branding record(s)")
        for brand in branding_data:
            print(f"   - {brand['code']}")
        
        # Upload Branding to tenant.stateInfo schema
        result_branding = uploader.create_mdms_data(
            schema_code='tenant.stateInfo',
            data_list=clean_nans(branding_data),
            tenant=CONFIG['state_tenant_id'],
            sheet_name='State_Branding'
        )
        
        # Summary
        if result_branding['failed'] == 0:
            print("\n✅ [SUCCESS] State Branding uploaded successfully!")
            print("\n🎉 PHASE 1 COMPLETED!")
            print("\n➡️  Proceed to PHASE 2: Common Master Data Upload")
        else:
            print("\n⚠️  [WARNING] Some branding records failed. Check errors/ folder.")

[PHASE 1] LOADING STATE BRANDING DATA

[INFO] Loaded 3 branding record(s)
   - pg
   - pg.citya
   - pg.cityb

[UPLOADING] tenant.stateInfo
   Tenant: pg
   Records: 3
   API URL: http://localhost:8094/mdms-v2/v2/_create/{schema_code}
   [EXISTS] [1/3] pg
   [EXISTS] [2/3] pg.citya
   [EXISTS] [3/3] pg.cityb
[SUMMARY] Created: 0
[SUMMARY] Already Exists: 3
[SUMMARY] Failed: 0

✅ [SUCCESS] State Branding uploaded successfully!

🎉 PHASE 1 COMPLETED!

➡️  Proceed to PHASE 2: Common Master Data Upload


---

# 🚀 PHASE 2: COMMON MASTER DATA

In Phase 2, you will:
1. **Enter Target Tenant ID** (e.g., `pg.citya` or `pg`)
2. **Upload Common Master Excel** with departments, designations, and complaint types
3. **Upload master data to API** - Creates departments, designations, and complaint types for the tenant

---

## Step 2.1: Enter Target Tenant & Upload Common Master

In [12]:
print("="*70)
print("        🎯  ENTER TARGET TENANT & UPLOAD COMMON MASTER")
print("="*70)
print()

# Tenant input widget
tenant_input = widgets.Text(
    value='',
    placeholder='e.g., pg, pg.citya, pg.cityb',
    description='🏛️ Tenant ID:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

# File upload widget
common_file_upload = widgets.FileUpload(
    accept='.xlsx,.xls',
    multiple=False,
    description='Select File:',
    layout=widgets.Layout(width='70%')
)

# Combined upload button
upload_btn = widgets.Button(
    description='⬆️ Set Tenant & Upload Common Master',
    button_style='success',
    layout=widgets.Layout(width='95%', height='40px')
)

upload_output = widgets.Output()

def on_upload(b):
    global COMMON_MASTER_FILE, SELECTED_TENANT
    
    with upload_output:
        clear_output()
        
        # Validate tenant input
        if not tenant_input.value.strip():
            print("❌ Please enter a tenant ID")
            return
        
        # Validate file selection
        if not common_file_upload.value:
            print("❌ Please select Common Master Excel file")
            return
        
        # Set tenant
        SELECTED_TENANT = tenant_input.value.strip().lower()
        
        # Save file
        os.makedirs('upload', exist_ok=True)
        uploaded_file = common_file_upload.value[0]
        content = uploaded_file['content']
        filename = 'Common_Master.xlsx'
        
        upload_path = os.path.join('upload', filename)
        with open(upload_path, 'wb') as f:
            f.write(content)
        
        print("="*70)
        print("  📋 VALIDATING FILE AGAINST MDMS SCHEMA...")
        print("="*70)
        
        # Validate the file against MDMS schema
        try:
            from mdms_validator import MDMSValidator
            mdms_validator = MDMSValidator()
            result = mdms_validator.validate_excel_file(
                excel_file=upload_path,
                tenant_id=SELECTED_TENANT,
                schema_code='common.master'
            )
            
            if result['valid']:
                print("\n✅ MDMS VALIDATION PASSED!")
                print("   All required fields match MDMS schema.")
                COMMON_MASTER_FILE = upload_path
                
                print("\n" + "="*70)
                print("  ✅ FILE UPLOADED SUCCESSFULLY")
                print("="*70)
                print(f"\n🎯 Target Tenant: {SELECTED_TENANT}")
                print(f"\n📄 File: {upload_path}")
                print("\n➡️  Proceed to load and upload departments & designations")
                print("="*70)
            else:
                print("\n❌ MDMS VALIDATION FAILED!")
                print(f"   Found {len(result['errors'])} error(s):\n")
                
                # Show first 5 errors
                for i, error in enumerate(result['errors'][:5], 1):
                    print(f"   Error {i}:")
                    if 'row' in error:
                        print(f"      Sheet: {error.get('sheet', 'Unknown')}")
                        print(f"      Row: {error['row']}")
                        if 'column' in error:
                            print(f"      Column: {error['column']}")
                        if 'value' in error:
                            print(f"      Value: {error.get('value', 'N/A')}")
                    print(f"      Message: {error['message']}")
                    print()
                
                if len(result['errors']) > 5:
                    print(f"   ... and {len(result['errors']) - 5} more errors")
                
                print("\n⚠️  Please fix the errors in your Excel file and try again.")
                print("="*70)
                COMMON_MASTER_FILE = None
                SELECTED_TENANT = None
                
        except Exception as e:
            print(f"\n⚠️  MDMS Validation error: {str(e)}")
            print("   Proceeding without validation...")
            COMMON_MASTER_FILE = upload_path
            print("\n" + "="*70)
            print("  ✅ FILE UPLOADED SUCCESSFULLY")
            print("="*70)
            print(f"\n🎯 Target Tenant: {SELECTED_TENANT}")
            print(f"\n📄 File: {upload_path}")
            print("\n➡️  Proceed to load and upload departments & designations")
            print("="*70)

upload_btn.on_click(on_upload)

# Show uploaded tenants as reference (if available)
if UPLOADED_TENANTS:
    reference_info = widgets.HTML(
        f"<p style='color: #666; background-color: #f0f0f0; padding: 10px; border-radius: 5px;'>"
        f"<b>📋 Reference - Uploaded Tenants from Phase 1:</b><br>"
        f"{', '.join(UPLOADED_TENANTS)}</p>"
    )
else:
    reference_info = widgets.HTML("")

# Display
upload_box = widgets.VBox([
    widgets.HTML("<h3>🎯 Step 2.1: Enter Target Tenant & Upload Common Master</h3>"),
    widgets.HTML("<p style='color: #666;'><i>Enter the tenant ID and upload the Common Master Excel file</i></p>"),
    reference_info,
    widgets.HTML("<br>"),
    tenant_input,
    widgets.HTML("<p style='font-size: 12px; color: #888;'><i>💡 You can upload to any tenant - it doesn't have to be from Phase 1</i></p>"),
    widgets.HTML("<br>"),
    widgets.HTML("<p style='color: #666;'><b>Select Common Master Excel File:</b></p>"),
    common_file_upload,
    widgets.HTML("<p style='font-size: 11px; color: #007bff;'><i>⚡ File will be validated against MDMS schema 'common.master'</i></p>"),
    widgets.HTML("<br>"),
    upload_btn,
    upload_output
])

display(upload_box)

        🎯  ENTER TARGET TENANT & UPLOAD COMMON MASTER



In [13]:
print("="*70)
print("[PHASE 2] LOADING DEPARTMENTS & DESIGNATIONS")
print("="*70)

if not COMMON_MASTER_FILE:
    print("❌ Please upload Common Master Excel first!")
elif not SELECTED_TENANT:
    print("❌ Please select a tenant first!")
else:
    # Initialize reader
    reader = UnifiedExcelReader(COMMON_MASTER_FILE)
    uploader = APIUploader()
    
    # Read departments and designations (combined sheet)
    # Now also returns dept_name_to_code mapping for complaint types
    departments, designations, dept_loc, desig_loc, dept_name_to_code = reader.read_departments_designations(SELECTED_TENANT)
    
    # Create reverse mapping (code to name) for error Excel
    dept_code_to_name = {code: name for name, code in dept_name_to_code.items()}
    
    print(f"\n[INFO] Loaded {len(departments)} department(s)")
    for dept in departments:
        print(f"   - {dept['code']}: {dept['name']}")
    
    print(f"\n[INFO] Loaded {len(designations)} designation(s)")
    for desig in designations[:5]:
        print(f"   - {desig['code']}: {desig['name']} [{desig['departmentCode']}]")
    if len(designations) > 5:
        print(f"   ... and {len(designations) - 5} more")
    
    print(f"\n[INFO] Auto-generated {len(dept_loc) + len(desig_loc)} localization entries")
    print(f"\n[INFO] Created department name-to-code mapping: {dept_name_to_code}")
    
    # Store reverse mapping for error Excel
    uploader._dept_code_to_name = dept_code_to_name
    
    # Upload Departments (use combined sheet name for errors)
    result_dept = uploader.create_mdms_data(
        schema_code='common-masters.Department',
        data_list=clean_nans(departments),
        tenant=SELECTED_TENANT,
        sheet_name='Department_And_Designation'  # Combined sheet name
    )
    
    # Upload Designations (use same combined sheet name for errors)
    result_desig = uploader.create_mdms_data(
        schema_code='common-masters.Designation',
        data_list=clean_nans(designations),
        tenant=SELECTED_TENANT,
        sheet_name='Department_And_Designation'  # Same sheet name - will combine errors
    )
    
    # Upload Localization
    all_loc = dept_loc + desig_loc
    result_loc = uploader.create_localization_messages(
        localization_list=clean_nans(all_loc),
        tenant=SELECTED_TENANT,
        sheet_name='Dept_Desig_Localization'
    )
    
    # Summary
    if result_dept['failed'] == 0 and result_desig['failed'] == 0:
        print("\n✅ [SUCCESS] Departments & Designations uploaded successfully!")
        print("\n➡️  Proceed to upload Complaint Types")
    else:
        print("\n⚠️  [WARNING] Some items failed. Check errors/ folder.")

[PHASE 2] LOADING DEPARTMENTS & DESIGNATIONS

[INFO] Loaded 4 department(s)
   - DEPT_1: WATER DEPARTMENT
   - DEPT_2: ELECTRIC DEPARTMENT
   - DEPT_3: SANITATION DEPARTMENT
   - DEPT_4: ROAD DEPARTMENT

[INFO] Loaded 9 designation(s)
   - DESIG_01: Chief Engineer [DEPT_1]
   - DESIG_02: Assistant Engineer [DEPT_1]
   - DESIG_03: Supervisor [DEPT_1]
   - DESIG_04: Electrical Engineer [DEPT_2]
   - DESIG_05: Line Maintenance Engineer [DEPT_2]
   ... and 4 more

[INFO] Auto-generated 13 localization entries

[INFO] Created department name-to-code mapping: {'WATER DEPARTMENT': 'DEPT_1', 'ELECTRIC DEPARTMENT': 'DEPT_2', 'SANITATION DEPARTMENT': 'DEPT_3', 'ROAD DEPARTMENT': 'DEPT_4'}

[UPLOADING] common-masters.Department
   Tenant: pg
   Records: 4
   API URL: http://localhost:8094/mdms-v2/v2/_create/{schema_code}
   [OK] [1/4] DEPT_1
   [OK] [2/4] DEPT_2
   [OK] [3/4] DEPT_3
   [OK] [4/4] DEPT_4
[SUMMARY] Created: 4
[SUMMARY] Already Exists: 0
[SUMMARY] Failed: 0

[UPLOADING] common-maste

## Step 2.3: Load and Upload Complaint Types

In [14]:
print("="*70)
print("[PHASE 2] LOADING COMPLAINT TYPES")
print("="*70)

if not COMMON_MASTER_FILE:
    print("❌ Please upload Common Master Excel first!")
elif not SELECTED_TENANT:
    print("❌ Please select a tenant first!")
elif 'dept_name_to_code' not in locals():
    print("❌ Please run 'Load Departments & Designations' cell first!")
else:
    # Initialize reader
    reader = UnifiedExcelReader(COMMON_MASTER_FILE)
    uploader = APIUploader()
    
    # Read complaint types - pass dept_name_to_code mapping so it uses department codes
    complaint_types, ct_loc = reader.read_complaint_types(SELECTED_TENANT, dept_name_to_code)
    
    print(f"\n[INFO] Loaded {len(complaint_types)} complaint type(s)")
    for ct in complaint_types[:10]:
        dept_info = f" [Dept: {ct.get('department', 'N/A')}]" if ct.get('department') else ""
        print(f"   - {ct['serviceCode']}: {ct['name']}{dept_info}")
    if len(complaint_types) > 10:
        print(f"   ... and {len(complaint_types) - 10} more")
    
    print(f"\n[INFO] Auto-generated {len(ct_loc)} localization entries")
    
    # Upload Complaint Types (pass reverse mapping for error Excel)
    uploader._dept_code_to_name = dept_code_to_name
    result_ct = uploader.create_mdms_data(
        schema_code='RAINMAKER-PGR.ServiceDefs',
        data_list=clean_nans(complaint_types),
        tenant=SELECTED_TENANT,
        sheet_name='ComplaintTypes'
    )
    
    # Upload Localization
    result_loc = uploader.create_localization_messages(
        localization_list=clean_nans(ct_loc),
        tenant=SELECTED_TENANT,
        sheet_name='ComplaintTypes_Localization'
    )
    
    # Summary
    if result_ct['failed'] == 0:
        print("\n✅ [SUCCESS] Complaint Types uploaded successfully!")
        print("\n🎉 PHASE 2 COMPLETED!")
    else:
        print("\n⚠️  [WARNING] Some complaint types failed. Check errors/ folder.")

[PHASE 2] LOADING COMPLAINT TYPES

[INFO] Loaded 11 complaint type(s)
   - NoWaterSupply: No Water Supply [Dept: DEPT_1]
   - LowWaterPressure: Low Water Pressure [Dept: DEPT_1]
   - ContaminatedWater: Contaminated Water [Dept: DEPT_1]
   - PipelineLeakage: Pipeline Leakage [Dept: DEPT_1]
   - PowerOutage: Power Outage [Dept: DEPT_2]
   - StreetLightNotWorking: Street Light Not Working [Dept: DEPT_2]
   - TransformerIssue: Transformer Issue [Dept: DEPT_2]
   - PotholeOnRoad: Pothole on Road [Dept: DEPT_4]
   - RoadCave-in: Road Cave-in [Dept: DEPT_4]
   - GarbageNotCollected: Garbage Not Collected [Dept: DEPT_3]
   ... and 1 more

[INFO] Auto-generated 15 localization entries

[UPLOADING] RAINMAKER-PGR.ServiceDefs
   Tenant: pg
   Records: 11
   API URL: http://localhost:8094/mdms-v2/v2/_create/{schema_code}
   [OK] [1/11] NoWaterSupply
   [OK] [2/11] LowWaterPressure
   [OK] [3/11] ContaminatedWater
   [OK] [4/11] PipelineLeakage
   [OK] [5/11] PowerOutage
   [OK] [6/11] StreetLightNo

In [15]:
from IPython.display import display, HTML, FileLink
from datetime import datetime
import os

# Collect results
summary_data = []

if 'result_tenants' in locals():
    summary_data.append({
        'module': 'Tenants',
        'created': result_tenants.get('created', 0),
        'exists': result_tenants.get('exists', 0),
        'failed': result_tenants.get('failed', 0)
    })

if 'result_dept' in locals():
    summary_data.append({
        'module': 'Departments',
        'created': result_dept.get('created', 0),
        'exists': result_dept.get('exists', 0),
        'failed': result_dept.get('failed', 0)
    })

if 'result_desig' in locals():
    summary_data.append({
        'module': 'Designations',
        'created': result_desig.get('created', 0),
        'exists': result_desig.get('exists', 0),
        'failed': result_desig.get('failed', 0)
    })

if 'result_ct' in locals():
    summary_data.append({
        'module': 'Complaint Types',
        'created': result_ct.get('created', 0),
        'exists': result_ct.get('exists', 0),
        'failed': result_ct.get('failed', 0)
    })

# Calculate totals
total_created = sum(item['created'] for item in summary_data)
total_exists = sum(item['exists'] for item in summary_data)
total_failed = sum(item['failed'] for item in summary_data)
total_records = total_created + total_exists + total_failed

# Build HTML table
table_rows = []
for item in summary_data:
    total_for_module = item['created'] + item['exists'] + item['failed']
    table_rows.append(f"""
        <tr>
            <td style="padding: 8px; border: 1px solid #ddd;">{item['module']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; color: green; font-weight: bold;">{item['created']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; color: orange; font-weight: bold;">{item['exists']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; color: red; font-weight: bold;">{item['failed']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; font-weight: bold;">{total_for_module}</td>
        </tr>
    """)

timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# Check if error file exists
error_file_path = 'errors/FAILED_RECORDS.xlsx'
error_download_link = ''
if total_failed > 0 and os.path.exists(error_file_path):
    error_download_link = f"""
    <div style="margin: 20px 0; padding: 15px; background-color: #fff3cd; border-left: 4px solid #ffc107; border-radius: 5px;">
        <h3 style="margin-top: 0; color: #856404;"> Failed Records Detected</h3>
        <p style="color: #856404; margin-bottom: 10px;">
            <b>{total_failed} record(s)</b> failed to upload. Download the error Excel below to fix the issues:
        </p>
        <a href="{error_file_path}" download="FAILED_RECORDS.xlsx" 
           style="display: inline-block; padding: 10px 20px; background-color: #dc3545; color: white; 
                  text-decoration: none; border-radius: 5px; font-weight: bold;">
             Download Error Excel (FAILED_RECORDS.xlsx)
        </a>
        <p style="color: #856404; font-size: 12px; margin-top: 10px;">
            💡 <b>Instructions:</b> Open the file, fix the data columns, and re-upload. 
            Error columns (gray background) are protected and show the failure reason.
        </p>
    </div>
    """

html_content = f"""
<div style="font-family: Arial, sans-serif; padding: 20px; border: 2px solid #007bff; border-radius: 10px; background-color: #f8f9fa;">
    <h2 style="color: #007bff; margin-top: 0;">📊 Two-Phase Data Loader - Summary Report</h2>
    <p style="color: #666; margin-bottom: 20px;">Generated: {timestamp}</p>
    
    <table style="width: 100%; border-collapse: collapse; margin-bottom: 20px; background-color: white;">
        <thead>
            <tr style="background-color: #007bff; color: white;">
                <th style="padding: 12px; border: 1px solid #ddd; text-align: left;">Module</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;"> Created</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;"> Already Exists</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;"> Failed</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;"> Total</th>
            </tr>
        </thead>
        <tbody>
            {''.join(table_rows)}
            <tr style="background-color: #e9ecef; font-weight: bold;">
                <td style="padding: 12px; border: 1px solid #ddd;">TOTAL</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: green;">{total_created}</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: orange;">{total_exists}</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: red;">{total_failed}</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center;">{total_records}</td>
            </tr>
        </tbody>
    </table>
    
    <div style="display: flex; justify-content: space-around; margin: 20px 0;">
        <div style="text-align: center; padding: 15px; background-color: #d4edda; border-radius: 5px; flex: 1; margin: 0 5px;">
            <div style="font-size: 32px; font-weight: bold; color: #155724;">{total_created}</div>
            <div style="color: #155724;">Created</div>
        </div>
        <div style="text-align: center; padding: 15px; background-color: #fff3cd; border-radius: 5px; flex: 1; margin: 0 5px;">
            <div style="font-size: 32px; font-weight: bold; color: #856404;">{total_exists}</div>
            <div style="color: #856404;">Already Exists</div>
        </div>
        <div style="text-align: center; padding: 15px; background-color: #f8d7da; border-radius: 5px; flex: 1; margin: 0 5px;">
            <div style="font-size: 32px; font-weight: bold; color: #721c24;">{total_failed}</div>
            <div style="color: #721c24;">Failed</div>
        </div>
    </div>
    
    {error_download_link}
</div>
"""

display(HTML(html_content))

print("\n🎉 TWO-PHASE DATA UPLOAD COMPLETED!")

Module,Created,Already Exists,Failed,Total
Tenants,0,3,0,3
Departments,4,0,0,4
Designations,9,0,0,9
Complaint Types,11,0,0,11
TOTAL,24,3,0,27



🎉 TWO-PHASE DATA UPLOAD COMPLETED!
